In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append('%s/../' % os.getcwd())

import datetime
import pandas as pd
import numpy as np
import getpass

In [2]:
#importa as fórmulas do arquivo aluguel_offshore.py - note que este arquivo deverá estar na mesma pasta deste notebook
from aluguel_offshore import *
from formulas_auxiliares_ticker import *
from confirmacoes.aux_emails import email_sender

In [3]:
path = r'\\DRIOC0231PFS\Apoio_SB\Geral\BD\Aluguel_Offshore\Arquivos diários\out2018\consolidado\alguel_offshore_consolidado.xlsx'
save_path = r'\\DRIOC0231PFS\Apoio_SB\Geral\BD\Aluguel_Offshore\Arquivos diários\out2018\consolidado'

In [4]:
abertura_consol = pd.read_excel(path, sheetname = 'Custos Colsolidados')
abertura_wm_consol = pd.read_excel(path, sheetname = 'Custos WM Colsolidados',)
abertura_treas_ops_consol = pd.read_excel(path, sheetname = 'Custos TREAS OPS Colsolidados')

In [5]:
abertura_consol.columns = ['acc', 'book', 'currency', 'accrual']
abertura_consol.loc[:,'tipo'] = 'geral'
abertura_wm_consol.columns = ['acc', 'book', 'currency', 'accrual']
abertura_wm_consol.loc[:,'tipo'] = 'WM'
abertura_treas_ops_consol.columns = ['acc', 'book', 'currency', 'accrual']
abertura_treas_ops_consol.loc[:,'tipo'] = 'TREAS'

df = pd.concat([abertura_consol,abertura_wm_consol,abertura_treas_ops_consol], axis=0).reset_index(drop=True)
df = df[df['accrual']!=0]

In [6]:
df

,acc,book,currency,accrual,tipo
0,int_c,ACOES_OFFSHORE_.INTC,AUD,-3356.200000,geral
1,int_c,ACOES_OFFSHORE_.INTC,USD,323.205352,geral
2,int_c,ACOES_OFFSHORE_INTC,USD,-586.295352,geral
3,int_c,INTRADAY_PAIRS.INTC,USD,-50.878938,geral
4,int_c,LOCAL_ADR_INTC,USD,516.741270,geral
5,int_c,LOCAL_ADR_MR.INTC,USD,-590.172332,geral
6,ms_cayman,13F_COMM.CAYMAN,USD,-2616.350000,geral
7,ms_cayman,13F_CONS_CYCL.CAYMAN,USD,-1224.737759,geral
8,ms_cayman,13F_CONS_N_CYCL.CAYMAN,USD,-1826.935541,geral
9,ms_cayman,13F_ENERGY.CAYMAN,USD,-1775.570000,geral


In [7]:
df = df[(df['book']!='TREAS_OPS.CAYMAN')&(~(df['book'].str.contains('PRIVATE')|df['book'].str.contains('CLIENT')))]

#### Ajuste do output final

In [8]:
df_g = df.groupby(by=['acc', 'currency']).sum().reset_index()
#abertura_consol_grouped = abertura_consol.groupby(by=['acc', 'currency']).sum().reset_index()
#abertura_treas_ops_consol_grouped = abertura_treas_ops_consol.groupby(by=['acc', 'currency']).sum().reset_index()

In [9]:
info_emails = pd.DataFrame({
    'acc':['ms_cayman','ms_cayman','ubs_cayman','int_c','int_c'],
    'currency':['MXN','USD','MXN','USD','AUD'],
    'accrual_final':[-721885.03,-125142.35,-583686.48,-387.40,-3356.20]
})

In [10]:
info_emails

,acc,accrual_final,currency
0,ms_cayman,-721885.03,MXN
1,ms_cayman,-125142.35,USD
2,ubs_cayman,-583686.48,MXN
3,int_c,-387.40,USD
4,int_c,-3356.20,AUD


In [11]:
ajuste = pd.merge(df_g, info_emails, on=['acc','currency'], how='left')

In [12]:
ajuste

,acc,currency,accrual,accrual_final
0,int_c,AUD,-3356.200000,-3356.20
1,int_c,USD,-387.400000,-387.40
2,ms_cayman,MXN,-721855.030000,-721885.03
3,ms_cayman,USD,-122356.980000,-125142.35
4,ubs_cayman,MXN,-386503.452602,-583686.48
5,ubs_cayman,USD,87981.020814,NaN


In [13]:
ajuste.loc[:,'multiplicador'] = ajuste.loc[:,'accrual_final']/ajuste.loc[:,'accrual']

In [14]:
ajuste.loc[:,'diff'] = ajuste.loc[:,'accrual_final']-ajuste.loc[:,'accrual']

In [15]:
ajuste

,acc,currency,accrual,accrual_final,multiplicador,diff
0,int_c,AUD,-3356.200000,-3356.20,1.000000,-9.094947e-13
1,int_c,USD,-387.400000,-387.40,1.000000,-1.136868e-13
2,ms_cayman,MXN,-721855.030000,-721885.03,1.000042,-3.000000e+01
3,ms_cayman,USD,-122356.980000,-125142.35,1.022764,-2.785370e+03
4,ubs_cayman,MXN,-386503.452602,-583686.48,1.510171,-1.971830e+05
5,ubs_cayman,USD,87981.020814,NaN,NaN,NaN


In [16]:
df = pd.merge(df, ajuste.drop(['accrual_final','accrual'], axis=1), on=['acc','currency'], how='left')

In [17]:
df.loc[:,'accrual_final'] = df.loc[:,'accrual']*df.loc[:,'multiplicador']

In [18]:
df = df.drop(['accrual','multiplicador'], axis=1)

In [19]:
df_posi = df[df.loc[:,'accrual_final']>0].copy()
df_neg = df[df.loc[:,'accrual_final']<0].copy()

In [20]:
df_posi = df_posi.groupby(by=['acc','currency']).sum().reset_index().rename(columns={
    'accrual_final':'accrual_positivo'
})

In [21]:
df_neg_g = df_neg.groupby(by=['acc','currency']).sum().reset_index().rename(columns={'accrual_final':'accrual_agrupado'})

In [22]:
df_neg = pd.merge(df_neg, df_neg_g, on=['acc','currency'], how='left')

In [23]:
df_neg.loc[:,'pct'] = df_neg.loc[:,'accrual_final']/df_neg.loc[:,'accrual_agrupado']

In [24]:
df = pd.merge(df_neg, df_posi, on=['acc','currency'], how='left').fillna(0)

In [25]:
df.loc[:,'accrual'] = df.loc[:,'accrual_positivo']*df.loc[:,'pct']+df.loc[:,'accrual_final']

In [26]:
df = df.drop(['accrual_final','accrual_agrupado','pct','accrual_positivo'], axis=1)

In [27]:
check_final = pd.merge(df.groupby(by=['acc','currency']).sum().reset_index(), info_emails, on=['acc','currency'], how='left')

In [28]:
writer = pd.ExcelWriter(save_path+'\\Custos_de_Aluguel_Final.xlsx')

In [29]:
df.to_excel(writer, sheet_name='Custos Aluguel Offshore', index=False)
check_final.to_excel(writer, sheet_name='Tabela', index=False)

In [30]:
writer.save()

In [31]:
lista_files = [save_path+'\\Custos_de_Aluguel_Final.xlsx']

In [35]:
save_path

'\\\\DRIOC0231PFS\\Apoio_SB\\Geral\\BD\\Aluguel_Offshore\\Arquivos diários\\out2018\\consolidado'

In [32]:
body = '''Legenda: 
    Aba Custos Aluguel Offshore contem os custos consolidados
        Tipo Geral: books Asset
        Tipo TREAS: book TREAS_OPS
        Tipo WM: Custos dos clients WM
    Aba Tabela contém os custos somados por Account e por moeda
        accrual: Dados distribuídos
        accrual_final: custos debitados da conta em cayman'''

In [33]:
send_to = 'OL-equity-backoffice@btgpactual.com;OL-EQUITYBD@btgpactual.com;DL-Finance-EQUITIES@btgpactual.onmicrosoft.com;\
DL-Finance-BUC-Colombia@btgpactual.com;DL-Finance-Trading@btgpactual.com;OL-Finance-Trading@btgpactual.com;\
Alexandre.Tex@btgpactual.com;Joao.Seidel@btgpactual.com;Valesca.Cidronio@btgpactual.com;Rafael.Barbirato@btgpactual.com;\
DL-DataOffice@btgpactual.com'

In [34]:
mes = 'October'

In [36]:
email_sender('OL-EQUITYBD@btgpactual.com',send_to,'Stock Loan | Monthly Costs  - %s' %mes, body, lista_files)